In [23]:
import pandas as pd
from datetime import datetime, timedelta
import pytz
import numpy as np
import MetaTrader5 as mt5

symbol = ('ETHUSD')
lots = 50.0
timezone = pytz.timezone("Etc/UTC")
start_n_minutes_ago = 1440
start_from = datetime.now() - timedelta(minutes=start_n_minutes_ago)
start_pos = 0
until_now = datetime.now()

mt5.initialize()

login = 1051286908
password = "G6NEWUYDL5"
server = "FTMO-Demo"

mt5.login(login, password, server)

if mt5.login(login, password, server):
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(
                account, mt5.last_error()
            )
        )

def gather_data(symbol, start_from):
    rates = mt5.copy_rates_from_pos(
        symbol, mt5.TIMEFRAME_M5, start_pos, start_n_minutes_ago
    )
    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")

    df['ha_open'] = (df.close.shift(1) + df.close.shift(2))/2
    df['ha_close'] = (df.open+df.low+df.close+df.high)/4

    g_candle = df.open < df.close
    df['green_candle'] = g_candle

    df['range_price'] = df.close-df.open
    df['range_price_movement'] = df.range_price.shift(1)

    df['ha_range_price'] = df.ha_close-df.ha_open
    df['ha_range_price_movement'] = df.ha_range_price.shift(1)

    df['median_price'] = (df.high+df.low)/2

    momentum_up = 'Ascending'
    momentum_down = 'Descending'

    def conditions(df):
        if (df['range_price'] > 1) & (df['range_price_movement'] > 1):
            return momentum_up
        elif (df['range_price'] < -1) & (df['range_price_movement'] < -1):
            return momentum_down

    df['momentum'] = df.apply(conditions, axis=1)
    df.loc[df['range_price'] >= 4,
           'candle_clasification'] = 'Significant G Candle'
    df.loc[df['range_price'] <= -4,
           'candle_clasification'] = 'Significant R Candle'

    df['rel_volume'] = df.tick_volume > 400

    median_move_up = 'Increasing'
    median_move_down = 'Decreasing'
    df['median_price_trend'] = df.median_price.shift(5)

    def median_move(df):
        if (df['median_price'] > df['median_price_trend']):
            return median_move_up
        elif (df['median_price'] < df['median_price_trend']):
            return median_move_down
    df['median_price_move'] = df.apply(median_move, axis=1)

    df.loc[df['range_price'] >= 7, 'abnormal_candle'] = 'Abnormal G Candle'
    df.loc[df['range_price'] <= -7, 'abnormal_candle'] = 'Abnormal R Candle'

    long = 'LONG'
    short = 'SHORT'
    none = 'No Entry'
    
    df['v_price'] = df['range_price']*df['tick_volume']
    
    def checking(df):
        if (df['rel_volume']) & (df['momentum'] == 'Ascending') & (df['candle_clasification'] == 'Significant G Candle') | (df['abnormal_candle'] == 'Abnormal G Candle'):
            return long
        elif (df['rel_volume']) & (df['momentum'] == 'Descending') & (df['candle_clasification'] == 'Significant R Candle') | (df['abnormal_candle'] == 'Abnormal R Candle'):
            return short
        else:
            return none
    df['entry_type'] = df.apply(checking, axis=1)

    df['mean'] = df.close.mean()
    df['ha_range_price_movement'] = df.ha_range_price.mean()
    df['std'] = df.ha_range_price.std()

    df.loc[df['entry_type'] == 'LONG', 'entry_price'] = df.close
    df.loc[df['entry_type'] == 'SHORT', 'entry_price'] = df.close

    df.loc[df['entry_type'] == 'LONG', 'sl'] = df.open
    df.loc[df['entry_type'] == 'SHORT', 'sl'] = df.open

    df.loc[df['entry_type'] == 'LONG', 'tp'] = df.close + 10
    df.loc[df['entry_type'] == 'SHORT', 'tp'] = df.close - 10

    return df


  login=1051286908
  trade_mode=0
  leverage=100
  limit_orders=200
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=45493.5
  credit=0.0
  profit=0.0
  equity=45493.5
  margin=0.0
  margin_free=45493.5
  margin_level=0.0
  margin_so_call=100.0
  margin_so_so=30.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=FTMO Free Trial USD
  server=FTMO-Demo
  currency=USD
  company=FTMO S.R.O.


In [24]:
df = gather_data(f'{symbol}',start_from)
df.to_csv(r"D:\Trading\Bot/ETHUSD.csv",index=False)

In [25]:
df.head(10)

,time,open,high,low,close,tick_volume,spread,real_volume,ha_open,ha_close,...,median_price_trend,median_price_move,abnormal_candle,v_price,entry_type,mean,std,entry_price,sl,tp
0,2022-10-05 21:45:00,1349.32,1350.68,1348.11,1349.25,520,170,0,NaN,1349.3400,...,NaN,None,NaN,-36.40,No Entry,1333.368444,1.464503,NaN,NaN,NaN
1,2022-10-05 21:50:00,1349.24,1352.89,1347.32,1351.56,567,170,0,NaN,1350.2525,...,NaN,None,NaN,1315.44,No Entry,1333.368444,1.464503,NaN,NaN,NaN
2,2022-10-05 21:55:00,1351.53,1356.56,1351.53,1356.56,531,170,0,1350.405,1354.0450,...,NaN,None,NaN,2670.93,LONG,1333.368444,1.464503,1356.56,1351.53,1366.56
3,2022-10-05 22:00:00,1356.48,1363.89,1355.86,1359.73,744,170,0,1354.060,1358.9900,...,NaN,None,NaN,2418.00,No Entry,1333.368444,1.464503,NaN,NaN,NaN
4,2022-10-05 22:05:00,1359.74,1360.11,1357.67,1358.35,215,170,0,1358.145,1358.9675,...,NaN,None,NaN,-298.85,No Entry,1333.368444,1.464503,NaN,NaN,NaN
5,2022-10-06 00:05:00,1347.14,1347.24,1346.25,1346.92,243,170,0,1359.040,1346.8875,...,1349.395,Decreasing,NaN,-53.46,No Entry,1333.368444,1.464503,NaN,NaN,NaN
6,2022-10-06 00:10:00,1346.91,1347.02,1345.62,1345.63,330,170,0,1352.635,1346.2950,...,1350.105,Decreasing,NaN,-422.40,No Entry,1333.368444,1.464503,NaN,NaN,NaN
7,2022-10-06 00:15:00,1345.66,1346.47,1344.81,1346.24,210,170,0,1346.275,1345.7950,...,1354.045,Decreasing,NaN,121.80,No Entry,1333.368444,1.464503,NaN,NaN,NaN
8,2022-10-06 00:20:00,1346.29,1348.47,1346.29,1347.68,182,170,0,1345.935,1347.1825,...,1359.875,Decreasing,NaN,252.98,No Entry,1333.368444,1.464503,NaN,NaN,NaN
9,2022-10-06 00:25:00,1347.66,1347.76,1347.22,1347.58,69,170,0,1346.960,1347.5550,...,1358.890,Decreasing,NaN,-5.52,No Entry,1333.368444,1.464503,NaN,NaN,NaN
